<center><a href="https://www.featuretools.com/"><img src="img/featuretools-logo.png" width="400" height="200" /></a></center>

<h2> A Featuretools Baseline </h2>
<p>The following tutorial illustrates a featuretools baseline model for the NYC Taxi Trip Duration competition on Kaggle. This notebook follows the structure of the previous worksheet, but uses deep feature synthesis to create the model.</p>

<h2>Step 1: Download raw data </h2>
<p>As always, if you have not yet downloaded the data it can be found at the <a href="https://www.kaggle.com/c/nyc-taxi-trip-duration/data">Kaggle website</a>. After installing featuretools following <a href = "https://docs.featuretools.com/">the instructions in the documentation</a> you can run the following.
</p>


In [1]:
import pandas as pd
import numpy as np
import featuretools as ft
import taxi_utils
ft.__version__

'0.5.1'

In [2]:
TRAIN_DIR = "data/train.csv"
TEST_DIR = "data/test.csv"

data_train, data_test = taxi_utils.read_data(TRAIN_DIR, TEST_DIR)

data_train.head(5)

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,False,455
1,id2377394,1,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,False,663
2,id3858529,2,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,False,2124
3,id3504673,2,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,False,429
4,id2181028,2,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,False,435


<h2>Step 2: Prepare the Data </h2>
<p>Let's create another column to define test and train datasets.</p>

In [3]:
data_train['test_data'] = False
data_test['test_data'] = True

<p>We can now combine the data. </p>

In [4]:
data = pd.concat([data_train, data_test], sort=True)

<h2>Step 3: Create baseline features using DFS </h2>
<p>Instead of manually creating features, such as month of <b>pickup_datetime</b>, we can let featuretools come up with them. </p>

<p>Within featuretools there is a standard format for representing data that is used to set up predictions and build features. A <b>EntitySet</b> stores information about entities (database table), variables (columns in database tables), relationships, and the data itself. </p>

<p> First, we create the EntitySet.</p>

In [5]:
es = ft.EntitySet("taxi")

<p>We can then use the `entity_from_dataframe` method to add an Entity called <i>trips</i>. We want to track the `id`, the `time_index` and specify other types of variables we care about in this entity. </p>

<p>As a note: Featuretools will try to interpret the types of variables. We can override this interpretation by specifying the types. In this case, I wanted <b>passenger_count</b> to be a type of Ordinal, and <b>vendor_id</b> to be of type Categorical.</p>

In [6]:
from featuretools import variable_types as vtypes

trip_variable_types = {
    'passenger_count': vtypes.Ordinal, 
    'vendor_id': vtypes.Categorical,
}

es.entity_from_dataframe(entity_id="trips",
                         dataframe=data,
                         index="id",
                         time_index='pickup_datetime',
                         variable_types=trip_variable_types)

Entityset: taxi
  Entities:
    trips [Rows: 2050266, Columns: 11]
  Relationships:
    No relationships

In [7]:
es['trips'].df

,id,dropoff_latitude,dropoff_longitude,pickup_datetime,pickup_latitude,pickup_longitude,store_and_fwd_flag,test_data,trip_duration,passenger_count,vendor_id
id0190469,id0190469,40.829182,-73.938828,2016-01-01 00:00:17,40.719158,-73.981743,False,False,849.0,5,2
id0621643,id0621643,40.769379,-73.969330,2016-01-01 00:00:22,40.716881,-73.981850,False,True,NaN,2,2
id1384355,id1384355,40.891788,-73.854263,2016-01-01 00:00:28,40.733562,-73.976501,False,True,NaN,1,1
id1665586,id1665586,40.717491,-73.958038,2016-01-01 00:00:53,40.747166,-73.985085,False,False,1294.0,1,1
id1210365,id1210365,40.815170,-73.947479,2016-01-01 00:01:01,40.801041,-73.965279,False,False,408.0,5,2
id3888279,id3888279,40.750340,-73.991341,2016-01-01 00:01:14,40.751331,-73.982292,False,False,280.0,1,1
id0924227,id0924227,40.742989,-73.989357,2016-01-01 00:01:20,40.759800,-73.970108,False,False,736.0,1,1
id2568735,id2568735,40.748665,-73.876602,2016-01-01 00:01:24,40.759865,-73.972267,False,True,NaN,2,1
id2294362,id2294362,40.847771,-73.936493,2016-01-01 00:01:33,40.773891,-73.984993,False,False,712.0,1,2
id1078247,id1078247,40.761734,-73.974854,2016-01-01 00:01:37,40.764072,-73.973335,False,False,114.0,1,2


<p>We can also normalize some of the columns to create new entities. So a <i>vendors</i> entity is created based on the unique values in the <i>vendor_id</i> column in <i>trips</i>.</p>

In [8]:
es.normalize_entity(base_entity_id="trips",
                    new_entity_id="vendors",
                    index="vendor_id")

es.normalize_entity(base_entity_id="trips",
                    new_entity_id="passenger_cnt",
                    index="passenger_count")

Entityset: taxi
  Entities:
    trips [Rows: 2050266, Columns: 11]
    vendors [Rows: 2, Columns: 2]
    passenger_cnt [Rows: 8, Columns: 2]
  Relationships:
    trips.vendor_id -> vendors.vendor_id
    trips.passenger_count -> passenger_cnt.passenger_count

<p>We can specify the time for each instance of the target_entity to calculate features. The timestamp represents the last time data can be used for calculating features by DFS. This is specified using a dataframe of cutoff times. Below we can see that the cutoff time for each trip is the pickup time.</p>

In [9]:
cutoff_time = es['trips'].df[['id', 'pickup_datetime']]

<p>Given this dataset, we would have about 2 million unique cutoff times. This is a good use case to use the approximate features parameter of DFS. In a large dataset, direct features that are aggregations on the prediction entity may not change much from cutoff time to cutoff time. Calculating the aggregation features at specific times every hour and using it for all cutoff times within the hour would save time and perhaps not lose much information. The approximate parameter in DFS lets you specify a window size to use when approximating these direct aggregation features.</p>

<p>We now create features using DFS.</p>

<b>Note, we can use an already calculated feature_matrix by doing the following:</b>
<p>You must copy and run the code.</p>
```python
feature_matrix = pd.read_csv('https://s3.amazonaws.com/featuretools-static/nyc_taxi/fm_simple.csv', 
                             index_col='id')
features = feature_matrix.columns.values
```

In [10]:
es.add_interesting_values()

trans_primitives = ['Minute', 'Hour', 'Day', 'Week', 'Month', 'Weekday', 'Is_weekend']

feature_matrix, features = ft.dfs(entityset=es,
                                  target_entity="trips",
                                  trans_primitives=trans_primitives,
                                  drop_contains=['trips.test_data'],
                                  verbose=True,
                                  cutoff_time=cutoff_time,
                                  approximate='36d')

Built 96 features
Elapsed: 06:40 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 11/11 chunks


<p>Here are the features created. Notice how some of the features match the manually created features in the previous notebook.</p>

In [11]:
print(len(features))

96


In [12]:
features[:25]

[<Feature: dropoff_latitude>,
 <Feature: dropoff_longitude>,
 <Feature: pickup_latitude>,
 <Feature: pickup_longitude>,
 <Feature: store_and_fwd_flag>,
 <Feature: test_data>,
 <Feature: trip_duration>,
 <Feature: passenger_count>,
 <Feature: vendor_id>,
 <Feature: MINUTE(pickup_datetime)>,
 <Feature: HOUR(pickup_datetime)>,
 <Feature: DAY(pickup_datetime)>,
 <Feature: WEEK(pickup_datetime)>,
 <Feature: MONTH(pickup_datetime)>,
 <Feature: WEEKDAY(pickup_datetime)>,
 <Feature: IS_WEEKEND(pickup_datetime)>,
 <Feature: vendors.SUM(trips.dropoff_latitude)>,
 <Feature: vendors.SUM(trips.dropoff_longitude)>,
 <Feature: vendors.SUM(trips.pickup_latitude)>,
 <Feature: vendors.SUM(trips.pickup_longitude)>,
 <Feature: vendors.SUM(trips.trip_duration)>,
 <Feature: vendors.STD(trips.dropoff_latitude)>,
 <Feature: vendors.STD(trips.dropoff_longitude)>,
 <Feature: vendors.STD(trips.pickup_latitude)>,
 <Feature: vendors.STD(trips.pickup_longitude)>]

<h2>Step 3: Build the Model </h2>

<p>We need to retrieve our labels for the train dataset, so we should merge our current feature matrix with the original dataset. </p>
<p>We also get the log of the trip duration so that a more linear relationship can be found.</p>

In [13]:
# separates the whole feature matrix into train data feature matrix, train data labels, and test data feature matrix 
X_train, labels, X_test = taxi_utils.get_train_test_fm(feature_matrix)
labels = np.log(labels.values + 1)

In [14]:
model = taxi_utils.train_xgb(X_train, labels)

[0]	train-rmse:5.00535	valid-rmse:5.00507
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 60 rounds.
[10]	train-rmse:1.00739	valid-rmse:1.00793
[20]	train-rmse:0.575585	valid-rmse:0.577753
[30]	train-rmse:0.5238	valid-rmse:0.527231
[40]	train-rmse:0.49523	valid-rmse:0.500017
[50]	train-rmse:0.475479	valid-rmse:0.481395
[60]	train-rmse:0.463175	valid-rmse:0.470005
[70]	train-rmse:0.453555	valid-rmse:0.461199
[80]	train-rmse:0.448871	valid-rmse:0.457243
[90]	train-rmse:0.445029	valid-rmse:0.453938
[100]	train-rmse:0.436242	valid-rmse:0.445635
[110]	train-rmse:0.434033	valid-rmse:0.443912
[120]	train-rmse:0.425273	valid-rmse:0.435776
[130]	train-rmse:0.422681	valid-rmse:0.433628
[140]	train-rmse:0.417246	valid-rmse:0.428812
[150]	train-rmse:0.415018	valid-rmse:0.427003
[160]	train-rmse:0.412853	valid-rmse:0.425193
[170]	train-rmse:0.407265	valid-rmse:0.420134
[180]	train-rmse:0.401503	valid-rmse:0.414888

<h2>Step 4: Make a Submission </h2>

In [15]:
submission = taxi_utils.predict_xgb(model, X_test)
submission.head(5)

,trip_duration
id,
id0621643,936.572571
id1384355,1832.627319
id2568735,1458.195801
id3700764,1253.478516
id3008929,287.928894


In [16]:
submission.to_csv('trip_duration_ft_simple.csv', index=True, index_label='id')

<dt>This solution:</dt>
<dd>&nbsp; &nbsp; Received a score of 0.45288 on the Kaggle competition.</dd>
<dd>&nbsp; &nbsp; Placed 685 out of 1257.</dd>
<dd>&nbsp; &nbsp; Beat 45% of competitors on the Kaggle competition.</dd>
<dd>&nbsp; &nbsp; Scored 4% better than the baseline solution</dd>
<dd>&nbsp; &nbsp; Had a modeling RMSLE of 0.40196</dd>

December 27, 2017.

<h2>Additional Analysis</h2>
<p>Let's look at how important each feature was for the model.</p>

In [17]:
feature_names = X_train.columns.values
ft_importances = taxi_utils.feature_importances(model, feature_names)
ft_importances

,feature_name,importance
4,dropoff_latitude,8287.0
0,dropoff_longitude,7675.0
9,pickup_latitude,6207.0
3,pickup_longitude,5678.0
7,HOUR(pickup_datetime),2981.0
18,MINUTE(pickup_datetime),2111.0
27,DAY(pickup_datetime),2094.0
34,WEEK(pickup_datetime),1578.0
15,WEEKDAY(pickup_datetime),1356.0
10,IS_WEEKEND(pickup_datetime),400.0


<p>
    <img src="https://www.featurelabs.com/wp-content/uploads/2017/12/logo.png" alt="Featuretools" />
</p>

Featuretools was created by the developers at [Feature Labs](https://www.featurelabs.com/). If building impactful data science pipelines is important to you or your business, please [get in touch](https://www.featurelabs.com/contact/).